In [5]:
!pip install torch psutil tqdm numpy rogue-score


ERROR: Could not find a version that satisfies the requirement rogue-score (from versions: none)
ERROR: No matching distribution found for rogue-score

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import json
from typing import List, Dict, Optional
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Config, GPT2Model, GPT2TokenizerFast, GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm
import os
import psutil
import gc
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt


def log_to_file(message):
    """Helper function for logging"""
    print(message)

class BaselineGPT2MultiTask(nn.Module):
    def __init__(self, num_intents: int, num_categories: int, num_ner_labels: int,
                 dropout_rate: float, loss_weights: Optional[Dict[str, float]] = None,
                 ner_class_weights: Optional[torch.Tensor] = None,
                 category_class_weights: Optional[torch.Tensor] = None,
                 intent_class_weights: Optional[torch.Tensor] = None):
        super().__init__()
        log_to_file("Initializing classification model...")
        self.config = GPT2Config.from_pretrained('gpt2')
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        hidden_size = self.config.n_embd

        for param in self.gpt2.parameters():
            param.requires_grad = False
        log_to_file("All GPT-2 layers remain frozen")

        self.intent_head = nn.Sequential(
            nn.Linear(hidden_size, hidden_size), nn.Tanh(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_intents)
        )
        self.category_head = nn.Sequential(
            nn.Linear(hidden_size, hidden_size), nn.Tanh(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_categories)
        )
        self.ner_head = nn.Sequential(
            nn.Linear(hidden_size, hidden_size), nn.Tanh(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_ner_labels)
        )

        self.loss_weights = loss_weights or {'intent': 0.3, 'category': 0.3, 'ner': 0.4}
        self.intent_loss_fn = nn.CrossEntropyLoss(weight=intent_class_weights) if intent_class_weights is not None else nn.CrossEntropyLoss()
        self.category_loss_fn = nn.CrossEntropyLoss(weight=category_class_weights) if category_class_weights is not None else nn.CrossEntropyLoss()
        self.ner_loss_fn = nn.CrossEntropyLoss(weight=ner_class_weights) if ner_class_weights is not None else nn.CrossEntropyLoss()

        log_to_file(f"Classification model initialized with loss weights: {self.loss_weights}")

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor,
                intent_labels: Optional[torch.Tensor] = None,
                category_labels: Optional[torch.Tensor] = None,
                ner_labels: Optional[torch.Tensor] = None) -> Dict[str, torch.Tensor]:
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        masked_features = sequence_output * attention_mask.unsqueeze(-1)
        sequence_repr = masked_features.sum(dim=1) / attention_mask.sum(dim=1, keepdim=True)

        intent_logits = self.intent_head(sequence_repr)
        category_logits = self.category_head(sequence_repr)
        ner_logits = self.ner_head(sequence_output)

        output_dict = {
            'intent_logits': intent_logits,
            'category_logits': category_logits,
            'ner_logits': ner_logits
        }

        if all(label is not None for label in [intent_labels, category_labels, ner_labels]):
            intent_loss = self.intent_loss_fn(intent_logits, intent_labels)
            category_loss = self.category_loss_fn(category_logits, category_labels)
            active_loss = attention_mask.view(-1) == 1
            active_logits = ner_logits.view(-1, ner_logits.size(-1))[active_loss]
            active_labels = ner_labels.view(-1)[active_loss]
            ner_loss = self.ner_loss_fn(active_logits, active_labels)

            total_loss = (self.loss_weights['intent'] * intent_loss +
                          self.loss_weights['category'] * category_loss +
                          self.loss_weights['ner'] * ner_loss)

            output_dict.update({
                'loss': total_loss,
                'intent_loss': intent_loss,
                'category_loss': category_loss,
                'ner_loss': ner_loss
            })
        return output_dict
    
def inference_baseline(model, text, tokenizer, label_encoders, max_length, device):
        """Run inference with the BaselineGPT2MultiTask model"""
        model.eval()
        
        # Tokenize input
        encoding = tokenizer(
            text,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        inputs = {
            "input_ids": encoding["input_ids"].to(device),
            "attention_mask": encoding["attention_mask"].to(device)
        }

        # Run model
        with torch.no_grad():
            outputs = model(**inputs)

        # Get softmax probabilities for confidence scores
        intent_logits = outputs["intent_logits"]
        category_logits = outputs["category_logits"]
        ner_logits = outputs["ner_logits"]

        # Apply softmax to get probabilities
        intent_probs = torch.nn.functional.softmax(intent_logits, dim=-1)[0]
        category_probs = torch.nn.functional.softmax(category_logits, dim=-1)[0]
        ner_probs = torch.nn.functional.softmax(ner_logits, dim=-1)

        # Get top predictions and their confidences
        intent_pred = torch.argmax(intent_probs).cpu().item()
        intent_confidence = intent_probs[intent_pred].cpu().item()  # As decimal (0.0-1.0)

        category_pred = torch.argmax(category_probs).cpu().item()
        category_confidence = category_probs[category_pred].cpu().item()  # As decimal (0.0-1.0)

        ner_preds = torch.argmax(ner_probs, dim=-1).cpu().numpy()[0]
        ner_confidences = torch.max(ner_probs, dim=-1)[0][0].cpu().numpy()  # Get max probability for each token

        # Map to labels
        intent_decoder = {v: k for k, v in label_encoders["intent_encoder"].items()}
        category_decoder = {v: k for k, v in label_encoders["category_encoder"].items()}
        ner_decoder = {v: k for k, v in label_encoders["ner_label_encoder"].items()}

        intent_label = intent_decoder[intent_pred]
        category_label = category_decoder[category_pred]

        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].tolist())
        seq_len = int(inputs["attention_mask"][0].sum().item())
        ner_labels = [ner_decoder[pred] for pred in ner_preds[:seq_len]]

        # Extract entities from NER labels with confidence
        entities = []
        current_entity = None
        entity_tokens = []
        entity_confidences = []
        entity_type = None

        for i, (token, label, confidence) in enumerate(zip(tokens[:seq_len], ner_labels, ner_confidences[:seq_len])):
            if label.startswith("B-"):
                # If we were tracking an entity, save it before starting a new one
                if current_entity is not None:
                    entity_text = tokenizer.convert_tokens_to_string(entity_tokens).strip()
                    if entity_text:
                        # Calculate average confidence for the entity (as decimal)
                        avg_confidence = sum(entity_confidences) / len(entity_confidences)
                        entities.append({
                            "entity": entity_text,
                            "label": entity_type,
                            "confidence": avg_confidence
                        })

                # Start a new entity
                current_entity = label[2:]
                entity_type = label[2:]
                entity_tokens = [token]
                entity_confidences = [confidence]

            elif label.startswith("I-") and current_entity == label[2:]:
                # Continue current entity
                entity_tokens.append(token)
                entity_confidences.append(confidence)

            elif current_entity is not None:
                # End of an entity
                entity_text = tokenizer.convert_tokens_to_string(entity_tokens).strip()
                if entity_text:
                    # Calculate average confidence for the entity (as decimal)
                    avg_confidence = sum(entity_confidences) / len(entity_confidences)
                    entities.append({
                        "entity": entity_text,
                        "label": entity_type,
                        "confidence": avg_confidence
                    })
                current_entity = None
                entity_tokens = []
                entity_confidences = []
                entity_type = None

        # Check for unfinished entity
        if current_entity is not None:
            entity_text = tokenizer.convert_tokens_to_string(entity_tokens).strip()
            if entity_text:
                # Calculate average confidence for the entity (as decimal)
                avg_confidence = sum(entity_confidences) / len(entity_confidences)
                entities.append({
                    "entity": entity_text,
                    "label": entity_type,
                    "confidence": avg_confidence
                })

        return {
            "intent": {"label": intent_label, "confidence": intent_confidence},
            "category": {"label": category_label, "confidence": category_confidence},
            "ner": entities
        }


def generate_response(model, tokenizer, instruction, classification_result=None, max_length=512, device="cuda"):
    """Generate a response using GPT-2 model based on instruction and classification results"""
    model.eval()
    
    # Create a prompt for generating response
    if classification_result:
        intent = classification_result["intent"]["label"]
        category = classification_result["category"]["label"]
        entities_text = ", ".join([f"{entity['entity']} ({entity['label']})" for entity in classification_result["ner"]]) if classification_result["ner"] else "none"
        input_text = f"[INST] User query: {instruction}\nIntent: {intent}\nCategory: {category}\nEntities: {entities_text}\n\nProvide a helpful customer service response: [RESP]"
    else:
        input_text = f"[INST] User query: {instruction}\n\nProvide a helpful customer service response: [RESP]"
        
    # Tokenize input
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    attention_mask = torch.ones_like(input_ids).to(device)

    try:
        # Generate response
        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_beams=5,
                no_repeat_ngram_size=2,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode the generated text
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

        # Extract response part
        if "[RESP]" in generated_text:
            response = generated_text.split("[RESP]")[1].strip()
            # Clean up any trailing tokens
            if "[EOS]" in response:
                response = response.split("[EOS]")[0].strip()
        else:
            # Fallback extraction
            response = generated_text[len(input_text):].strip()
            
        # Format steps if present
        steps_pattern = re.search(r'(\d+)\.\s+([A-Z])', response)
        if steps_pattern or "step" in response.lower() or "follow" in response.lower():
            # Format steps to be on separate lines
            for i in range(1, 10):
                step_marker = f"{i}. "
                if step_marker in response and f"\n{i}. " not in response:
                    response = response.replace(step_marker, f"\n{i}. ")

            # Clean up any excess newlines
            response = re.sub(r'\n\s*\n', '\n\n', response)
            response = response.lstrip('\n')

        # Clean any technical artifacts
        response = re.sub(r'https?://\S+', '', response)  # Remove URLs
        response = re.sub(r'<[^>]*>', '', response)  # Remove HTML tags
        response = re.sub(r'\{\s*"[^"]*":', '', response)  # Remove JSON-like content
        response = re.sub(r'\s+', ' ', response).strip()  # Clean up whitespace

        return response

    except Exception as e:
        print(f"Error in generate_response: {e}")
        return f"I apologize, but I couldn't generate a response. Error: {str(e)}"

# Memory measurement functions
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Convert bytes to MB

def get_peak_memory_usage(func, *args, **kwargs):
    device_param = kwargs.pop('device', None)  # Remove device parameter before calling func
    
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
    
    start_mem = get_memory_usage()
    result = func(*args, **kwargs)  # Call without device parameter
    end_mem = get_memory_usage()
    
    peak_gpu_mem = 0
    if torch.cuda.is_available() and device_param == "cuda":
        peak_gpu_mem = torch.cuda.max_memory_allocated() / 1024 / 1024  # MB
        return result, peak_gpu_mem
    else:
        return result, max(0, end_mem - start_mem)
    
class BaselinePerformanceTest:
    def __init__(self, model_paths, test_data_path, num_samples=50):
        self.output_dir = model_paths["baseline_model_dir"]
        self.generation_model_path = model_paths["generation_model_path"]
        self.generation_tokenizer_path = model_paths["generation_tokenizer_path"]
        self.test_data_path = test_data_path
        self.num_samples = num_samples
        self.devices = ["cpu", "cuda"] if torch.cuda.is_available() else ["cpu"]
        self.results = {device: [] for device in self.devices}
        self.summary = {device: {"classification_time": [], "generation_time": [], "overall_time": [], "memory_usage": []} for device in self.devices}
    
    def load_models(self, device):
        print(f"\nLoading models on {device.upper()}...")
        
        # Load baseline model for classification
        encoders_path = os.path.join(self.output_dir, "label_encoders.json")
        hyperparams_path = os.path.join(self.output_dir, "hyperparameters.json")
        model_path = os.path.join(self.output_dir, "baseline_model.pth")
        
        print(f"Loading from: {encoders_path}")
        with open(encoders_path, 'r', encoding='utf-8') as f:
            self.label_encoders = json.load(f)
            
        with open(hyperparams_path, 'r', encoding='utf-8') as f:
            self.hyperparameters = json.load(f)
                
        self.gpt2_tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
        
        if self.gpt2_tokenizer.pad_token is None:
            self.gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
                
        self.classification_model = BaselineGPT2MultiTask(
            num_intents=len(self.label_encoders["intent_encoder"]),
            num_categories=len(self.label_encoders["category_encoder"]),
            num_ner_labels=len(self.label_encoders["ner_label_encoder"]),
            dropout_rate=self.hyperparameters["dropout_rate"]
        )
        
        if self.gpt2_tokenizer.pad_token_id is not None:
            self.classification_model.gpt2.resize_token_embeddings(len(self.gpt2_tokenizer))
        
        self.classification_model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
        self.classification_model.to(device)
        self.classification_model.eval()
        
        # Load generation model using from_pretrained instead of torch.load
        try:
            print(f"Attempting to load generation model from {self.generation_model_path}")
            self.generation_model = GPT2LMHeadModel.from_pretrained(self.generation_model_path).to(device)
            self.generation_tokenizer = GPT2Tokenizer.from_pretrained(self.generation_tokenizer_path)
            self.generation_tokenizer.pad_token = self.generation_tokenizer.eos_token
            self.generation_tokenizer.add_special_tokens({'additional_special_tokens': ['[INST]', '[RESP]', '[EOS]']})
            self.generation_model.resize_token_embeddings(len(self.generation_tokenizer))
        except Exception as e:
            print(f"Error loading generation model: {e}")
            print("Falling back to default GPT2...")
            self.generation_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
            self.generation_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            self.generation_tokenizer.pad_token = self.generation_tokenizer.eos_token
            self.generation_tokenizer.add_special_tokens({'additional_special_tokens': ['[INST]', '[RESP]', '[EOS]']})
            self.generation_model.resize_token_embeddings(len(self.generation_tokenizer))
        
        self.generation_model.eval()
        
        return self.classification_model, self.generation_model, self.gpt2_tokenizer, self.generation_tokenizer
    
    def run_benchmark(self):
        # Load test data
        with open(self.test_data_path, "r", encoding="utf-8") as f:
            test_data = json.load(f)
            
        # Select samples for testing
        if len(test_data) > self.num_samples:
            test_samples = test_data[:self.num_samples]
        else:
            test_samples = test_data
            
        print(f"Running performance test on {len(test_samples)} samples")
        
        # Run tests on each device
        for device in self.devices:
            print(f"\n{'='*80}")
            print(f"RUNNING BENCHMARK ON {device.upper()}")
            print(f"{'='*80}")
            
            classification_model, generation_model, gpt2_tokenizer, generation_tokenizer = self.load_models(device)
            
            for idx, sample in enumerate(tqdm(test_samples, desc=f"Testing on {device.upper()}")):
                instruction = sample["instruction"]
                
                # Measure classification time and memory
                def run_classification():
                    return inference_baseline(
                        classification_model, 
                        instruction, 
                        gpt2_tokenizer, 
                        self.label_encoders, 
                        self.hyperparameters["max_length"],
                        device
                    )
                
                classification_start = time.time()
                classification_result, classification_memory = get_peak_memory_usage(
                    run_classification, device=device
                )
                classification_end = time.time()
                classification_time = classification_end - classification_start
                
                # Create new input with classification results
                intent = classification_result["intent"]["label"]
                category = classification_result["category"]["label"]
                entities_text = ", ".join([f"{entity['entity']} ({entity['label']})" for entity in classification_result["ner"]]) if classification_result["ner"] else "none"
                new_input = f"{instruction} [Classified: Intent is '{intent}', Category is '{category}', Entities are {entities_text}]"
                
                # Measure generation time and memory
                def run_generation():
                    return generate_response(
                        generation_model,
                        generation_tokenizer,
                        instruction,
                        classification_result,
                        device=device
                    )
                
                generation_start = time.time()
                generated_response, generation_memory = get_peak_memory_usage(
                    run_generation, device=device
                )
                generation_end = time.time()
                generation_time = generation_end - generation_start
                
                # Calculate overall time and memory
                overall_time = classification_time + generation_time
                overall_memory = classification_memory + generation_memory
                
                # Store results
                result = {
                    "Model": "GPT2-Baseline Pipeline",
                    "Classification Time (s)": classification_time,
                    "Text Generation Time (s)": generation_time,
                    "Overall Inference Time (s)": overall_time,
                    "Device": "GPU" if device == "cuda" else "CPU",
                    "Memory Usage (MB)": overall_memory,
                    "Instruction": instruction,
                    "New Input": new_input,
                    "Generated Response": generated_response
                }
                self.results[device].append(result)
                
                # Update summary statistics
                self.summary[device]["classification_time"].append(classification_time)
                self.summary[device]["generation_time"].append(generation_time)
                self.summary[device]["overall_time"].append(overall_time)
                self.summary[device]["memory_usage"].append(overall_memory)
                
                # Print sample result
                print(f"\nSample {idx+1}/{len(test_samples)} (Device: {device.upper()}):")
                print(f"Instruction: {instruction}")
                print(f"New Input: {new_input}")
                print(f"Generated Response: {generated_response}")
                print(f"Model                | Classification Time (s)   | Text Generation Time (s)  | Overall Inference Time (s) | Device     | Memory Usage (MB)")
                print(f"GPT2-Baseline Pipeline| {classification_time:.4f}                    | {generation_time:.4f}                   | {overall_time:.4f}                   | {device.upper()}        | {overall_memory:.4f}")
            
            # Calculate average times
            self.summary[device]["avg_classification_time"] = sum(self.summary[device]["classification_time"]) / len(test_samples)
            self.summary[device]["avg_generation_time"] = sum(self.summary[device]["generation_time"]) / len(test_samples)
            self.summary[device]["avg_overall_time"] = sum(self.summary[device]["overall_time"]) / len(test_samples)
            self.summary[device]["avg_memory_usage"] = sum(self.summary[device]["memory_usage"]) / len(test_samples)
            
            print(f"\n{'='*80}")
            print(f"BENCHMARK SUMMARY FOR {device.upper()}")
            print(f"{'='*80}")
            print(f"Average Classification Time: {self.summary[device]['avg_classification_time']:.4f} seconds")
            print(f"Average Generation Time: {self.summary[device]['avg_generation_time']:.4f} seconds")
            print(f"Average Overall Time: {self.summary[device]['avg_overall_time']:.4f} seconds")
            print(f"Average Memory Usage: {self.summary[device]['avg_memory_usage']:.4f} MB")

        # Save results after all devices are benchmarked
        self.save_results()
        
    def save_results(self):
        """Save benchmark results to JSON files in the specified format"""
        
        # Save CPU results
        if "cpu" in self.devices and self.results["cpu"]:
            with open("performance_results_baseline_cpu.json", "w") as f:
                json.dump(self.results["cpu"], f, indent=4)
            print("CPU results saved to performance_results_baseline_cpu.json")
        
        # Save GPU results
        if "cuda" in self.devices and self.results["cuda"]:
            with open("performance_results_baseline_gpu.json", "w") as f:
                json.dump(self.results["cuda"], f, indent=4)
            print("GPU results saved to performance_results_baseline_gpu.json")
        
        # Create performance log and comparison plots
        self.create_performance_log()
        if "cpu" in self.devices and "cuda" in self.devices:
            self.create_comparison_plots()

    def create_performance_log(self):
        """Create a text file with performance comparisons"""
        with open("baseline_performance_log.txt", "w") as f:
            f.write("=== BASELINE PIPELINE PERFORMANCE BENCHMARK RESULTS ===\n\n")
            
            # Write CPU results
            if "cpu" in self.devices:
                f.write("CPU PERFORMANCE:\n")
                f.write(f"Average Classification Time: {self.summary['cpu']['avg_classification_time']:.4f} seconds\n")
                f.write(f"Average Generation Time: {self.summary['cpu']['avg_generation_time']:.4f} seconds\n")
                f.write(f"Average Overall Time: {self.summary['cpu']['avg_overall_time']:.4f} seconds\n")
                f.write(f"Average Memory Usage: {self.summary['cpu']['avg_memory_usage']:.4f} MB\n\n")
            
            # Write GPU results
            if "cuda" in self.devices:
                f.write("GPU PERFORMANCE:\n")
                f.write(f"Average Classification Time: {self.summary['cuda']['avg_classification_time']:.4f} seconds\n")
                f.write(f"Average Generation Time: {self.summary['cuda']['avg_generation_time']:.4f} seconds\n")
                f.write(f"Average Overall Time: {self.summary['cuda']['avg_overall_time']:.4f} seconds\n")
                f.write(f"Average Memory Usage: {self.summary['cuda']['avg_memory_usage']:.4f} MB\n\n")
            
            # Write comparison
            if "cpu" in self.devices and "cuda" in self.devices:
                f.write("CPU vs GPU COMPARISON:\n")
                class_speedup = self.summary['cpu']['avg_classification_time'] / self.summary['cuda']['avg_classification_time']
                gen_speedup = self.summary['cpu']['avg_generation_time'] / self.summary['cuda']['avg_generation_time']
                overall_speedup = self.summary['cpu']['avg_overall_time'] / self.summary['cuda']['avg_overall_time']
                
                f.write(f"Classification Speed Improvement: {class_speedup:.2f}x faster on GPU\n")
                f.write(f"Generation Speed Improvement: {gen_speedup:.2f}x faster on GPU\n")
                f.write(f"Overall Speed Improvement: {overall_speedup:.2f}x faster on GPU\n\n")
                
                f.write("Notes:\n")
                f.write("- Classification is typically faster than generation\n")
                f.write("- GPU provides greater speedup for generation than classification\n")
                f.write("- Memory usage is higher on GPU but processing is much faster\n")
            
            f.write("\n=== END OF REPORT ===\n")
        
        print("Performance log saved to baseline_performance_log.txt")

    def create_comparison_plots(self):
        """Create matplotlib comparison plots"""
        import matplotlib.pyplot as plt
        import numpy as np
        
        if "cpu" not in self.devices or "cuda" not in self.devices:
            print("Both CPU and GPU results are needed for comparison plots")
            return
        
        # Set up the plots
        plt.figure(figsize=(12, 12))
        
        # 1. Classification Time Comparison
        plt.subplot(3, 1, 1)
        
        labels = ['Classification', 'Generation', 'Overall']
        cpu_times = [self.summary['cpu']['avg_classification_time'], 
                    self.summary['cpu']['avg_generation_time'], 
                    self.summary['cpu']['avg_overall_time']]
        gpu_times = [self.summary['cuda']['avg_classification_time'],
                    self.summary['cuda']['avg_generation_time'],
                    self.summary['cuda']['avg_overall_time']]
        
        x = np.arange(len(labels))
        width = 0.35
        
        plt.bar(x - width/2, cpu_times, width, label='CPU')
        plt.bar(x + width/2, gpu_times, width, label='GPU')
        
        plt.xlabel('Task')
        plt.ylabel('Time (seconds)')
        plt.title('Average Time Comparison')
        plt.xticks(x, labels)
        plt.legend()
        
        # Add value labels on the bars
        for i, v in enumerate(cpu_times):
            plt.text(i - width/2, v + 0.01, f"{v:.2f}s", ha='center')
        
        for i, v in enumerate(gpu_times):
            plt.text(i + width/2, v + 0.01, f"{v:.2f}s", ha='center')
        
        # 2. Speedup Chart
        plt.subplot(3, 1, 2)
        
        speedup_labels = ['Classification', 'Generation', 'Overall']
        speedups = [
            self.summary['cpu']['avg_classification_time'] / self.summary['cuda']['avg_classification_time'],
            self.summary['cpu']['avg_generation_time'] / self.summary['cuda']['avg_generation_time'],
            self.summary['cpu']['avg_overall_time'] / self.summary['cuda']['avg_overall_time']
        ]
        
        plt.bar(speedup_labels, speedups, color='blue')
        plt.xlabel('Task')
        plt.ylabel('Speedup Factor (CPU time / GPU time)')
        plt.title('GPU Speedup Compared to CPU')
        
        # Add value labels on the bars
        for i, v in enumerate(speedups):
            plt.text(i, v + 0.1, f"{v:.2f}x", ha='center')
        
        # 3. Memory Usage
        plt.subplot(3, 1, 3)
        
        memory_labels = ['CPU', 'GPU']
        memory_usage = [self.summary['cpu']['avg_memory_usage'],
                    self.summary['cuda']['avg_memory_usage']]
        
        plt.bar(memory_labels, memory_usage, color='green')
        plt.xlabel('Device')
        plt.ylabel('Memory Usage (MB)')
        plt.title('Average Memory Usage')
        
        # Add memory usage labels
        for i, v in enumerate(memory_usage):
            plt.text(i, v + 0.1, f"{v:.1f} MB", ha='center')
        
        plt.tight_layout()
        plt.savefig('baseline_performance_comparison.png')
        plt.close()
        
        print("Performance comparison plots saved to baseline_performance_comparison.png")

# Move the main function OUTSIDE of the class
def main():
    # Define paths
    model_paths = {
        "baseline_model_dir": "../..//Baseline_freeze_v1",
        "generation_model_path": "../../text_generation_results_03-09-25/model",
        "generation_tokenizer_path": "../../text_generation_results_03-09-25/tokenizer"
    }
    
    test_data_path = "../../test.json"
    
    # Verify and print paths
    print(f"Baseline model dir: {model_paths['baseline_model_dir']}")
    print(f"Generation model path: {model_paths['generation_model_path']}")
    print(f"Test data path: {test_data_path}")
    
    # Number of samples to test
    num_samples = 50  # You can adjust this number
    
    # Check if CUDA is available
    device_info = "CUDA available" if torch.cuda.is_available() else "CUDA not available, using CPU only"
    print(device_info)
    
    # Run the benchmark
    benchmark = BaselinePerformanceTest(model_paths, test_data_path, num_samples)
    benchmark.run_benchmark()

if __name__ == "__main__":
    # Check available devices
    device_info = "CUDA" if torch.cuda.is_available() else "CPU"
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"Running on: {device_info}")
    
    # Run the benchmark
    main()
            

GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
Running on: CUDA
Baseline model dir: ../..//Baseline_freeze_v1
Generation model path: ../../text_generation_results_03-09-25/model
Test data path: ../../test.json
CUDA available
Running performance test on 50 samples

RUNNING BENCHMARK ON CPU

Loading models on CPU...
Loading from: ../..//Baseline_freeze_v1\label_encoders.json
Initializing classification model...
All GPT-2 layers remain frozen
Classification model initialized with loss weights: {'intent': 0.3, 'category': 0.3, 'ner': 0.4}
Attempting to load generation model from ../../text_generation_results_03-09-25/model


Testing on CPU:   2%|▏         | 1/50 [00:26<21:49, 26.72s/it]


Sample 1/50 (Device: CPU):
Instruction: i do not know what i have to do to get my invoice #85632
New Input: i do not know what i have to do to get my invoice #85632 [Classified: Intent is 'get_invoice', Category is 'invoice', Entities are # (refund_amount)]
Generated Response: i understand that you're unsure about the steps to take in order to retrieve your invoice with the number #{{money amount}}. let me guide you through the process. first, please log in to your account on our website. once logged in, navigate to the "billing" or "invoices" section. there, you should be able to find a list of your invoices. look for a tab or option that says "view invoice." click on that option, and you'll be directed to a page where you can view and download the invoice. if you encounter any difficulties or have further questions, feel free to reach out to our customer support team. they will be more than happy to assist you further.
Model                | Classification Time (s)   | Text Generati

Testing on CPU:   4%|▍         | 2/50 [00:39<14:55, 18.65s/it]


Sample 2/50 (Device: CPU):
Instruction: transfer to live agent
New Input: transfer to live agent [Classified: Intent is 'create_account', Category is 'account', Entities are live (account_type), agent (account_type)]
Generated Response: assuredly! i'm here to assist you in transferring to one of our {{account type}} agents. to get started, could you please provide me with your full name, email address, and a preferred username? once i have these details, i'll guide you through the process step by step, ensuring that you have a seamless experience. rest assured, your security and privacy are of utmost importance to us, so you can trust that your information will be handled with the highest level of care.
Model                | Classification Time (s)   | Text Generation Time (s)  | Overall Inference Time (s) | Device     | Memory Usage (MB)
GPT2-Baseline Pipeline| 0.4581                    | 12.5359                   | 12.9940                   | CPU        | 4.9922


...

Testing on CUDA:  98%|█████████▊| 49/50 [11:09<00:21, 21.60s/it]


Sample 49/50 (Device: CUDA):
Instruction: change  order ord-8335461
New Input: change  order ord-8335461 [Classified: Intent is 'track_order', Category is 'order', Entities are ord-8335461 (order_number)]
Generated Response: we understand that you would like to know the status of your order with the order number {{order number}}. to provide you with accurate information, could you please provide us with some additional details such as your name or email address associated with this order? this will enable us to locate the specific order and give you the most up-to-date status. thank you for your cooperation, and we look forward to assisting you further.
Model                | Classification Time (s)   | Text Generation Time (s)  | Overall Inference Time (s) | Device     | Memory Usage (MB)
GPT2-Baseline Pipeline| 0.7556                    | 34.2935                   | 35.0492                   | CUDA        | 2163.8970


Testing on CUDA: 100%|██████████| 50/50 [11:35<00:00, 13.92s/it]


Sample 50/50 (Device: CUDA):
Instruction: i need assistance to send some feedback for ur produucts
New Input: i need assistance to send some feedback for ur produucts [Classified: Intent is 'review', Category is 'feedback', Entities are produ (person_name)]
Generated Response: i'll take care of it! i'm here to assist you in sending your feedback about our products and services. your opinion matters to us, and we appreciate your willingness to share your thoughts. could you please provide me with more details about the specific product or service you would like to provide feedback on? this will help me guide you through the process more effectively.
Model                | Classification Time (s)   | Text Generation Time (s)  | Overall Inference Time (s) | Device     | Memory Usage (MB)
GPT2-Baseline Pipeline| 1.7705                    | 24.3571                   | 26.1275                   | CUDA        | 2158.7026

BENCHMARK SUMMARY FOR CUDA
Average Classification Time: 0.6331 seconds

Performance comparison plots saved to baseline_performance_comparison.png
